### 작업형1 예시문제
- 자동차 데이터 셋에서 qsec 컬럼을 Min-Max Scale로 변환 후 0.5보다 큰 값을 가지는 레코드(row) 수는?

In [7]:
import pandas as pd

df = pd.read_csv("mtcars.csv")

print(df['qsec'].head())
# 사용자 코딩
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
df['qsec'] = sc.fit_transform(df[['qsec']])

print(df['qsec'].head())

cond = df['qsec'] > 0.5

print(sum(cond)) # 9

0    16.46
1    17.02
2    18.61
3    19.44
4    17.02
Name: qsec, dtype: float64
0    0.233333
1    0.300000
2    0.489286
3    0.588095
4    0.300000
Name: qsec, dtype: float64
9


- len: 행의 수
- count(): NaN이 아닌 (컬럼별) 행의 수
- sum(): True(1)값을 더했기 때문에 조건문에 따라 달라짐

### 2번

In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

import pandas as pd

pd.set_option('display.max_columns', None)

train = pd.read_csv("data/customer_train.csv")
test = pd.read_csv("data/customer_test.csv")

# 사용자 코딩
# 1. EDA
# print(train.shape)
# print(test.shape)

# print(train.info()) # float64(3), int64(6), object(2)
# print(test.info()) # float64(3), int64(5), object(2)

# print(train.head(5))
# print(train.isnull().sum()) # 환불금액       2295
# print(test.isnull().sum()) # 환불금액       1611

# 2. 결측치 처리, 피처 엔지니어링
# target = train.pop('성별')
target = train.pop('성별')

train = train.drop('회원ID', axis=1)
test_id = test.pop('회원ID')

train['환불금액'] = train['환불금액'].fillna(0)
test['환불금액'] = test['환불금액'].fillna(0)

from sklearn.preprocessing import MinMaxScaler
cols = train.select_dtypes(exclude='O').columns

scaler = MinMaxScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

cols = train.select_dtypes(include='O').columns
# print(cols)
# print(set(train[cols]) == set(test[cols]))
# print(train[cols].nunique()) # 주구매상품    42 / 주구매지점    24
# print(test[cols].nunique()) # 주구매상품    41 / 주구매지점    24

# train = train.drop(cols, axis=1)
# test = test.drop(cols, axis=1)
# print(train.shape)
# print(test.shape)

from sklearn.preprocessing import LabelEncoder

for col in cols:
	le = LabelEncoder()
	train[col] = le.fit_transform(train[col])
	test[col] = le.transform(test[col])

# print(train.shape)
# print(test.shape)

# 3. train/test 나누기
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=12)
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 4. 모델 선택 학습 예측
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=12)
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
# print(pred[:,1])
# print(help(RandomForestClassifier.predict_proba))

# 5. 평가
from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_val, pred[:,1]))
# base line 0.596748188405797
# label encoder 0.6278170289855072
# label + minmax 0.6346195652173914

# 6. csv 생성
pred = model.predict_proba(test)
submit = pd.DataFrame({
	'pred': pred[:,1]
})

# print(submit.shape)
# print(submit.head())

submit.to_csv('result.csv', index=False)

# tt = pd.read_csv('result.csv')
# print(tt.head(5))
# 답안 제출 참고
# 아래 코드는 예시이며 변수명 등 개인별로 변경하여 활용
# pd.DataFrame변수.to_csv("result.csv", index=False)



### 3번

In [ ]:
# 출력을 원할 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

import pandas as pd
pd.set_option('display.max_columns', None)

df = pd.read_csv("data/Titanic.csv")
# print(df.head())

# 1번 Gender, Survived 변수 간의 독립성 검정, 카이제곱 검정 통계량
# 독립성 검정 - 표 or 로우 데이터
# print(df[['Gender', 'Survived']].head())

# 개수를 센 후, 크로스 탭으로 만들기
crosstab = pd.crosstab(df['Gender'], df['Survived'])
# print(crosstab.head())

# > Survived    0    1
# Gender
# female     81  233
# male      468  109

# 독립성 검정
from scipy.stats import chi2_contingency
# print(dir(scipy.stats))
# print(chi2_contingency(crosstab))

# 검정통계량: 260.71702016732104
# pvalue: 1.1973570627755645e-58
# (260.71702016732104, 1.1973570627755645e-58, 1, array([[193.47474747, 120.52525253], [355.52525253, 221.47474747]]))

result = 260.71702016732104
# print(round(result, 3))

# 1번 답: 260.717

# 2번 로지스틱 회귀모형(분류 모델) -> Parch 변수 계수 값
cols = ['Gender', 'SibSp', 'Parch', 'Fare']
# print(df[cols].head())

from statsmodels.formula.api import logit

model = logit('Survived ~ C(Gender) + SibSp + Parch + Fare', data=df).fit()
# print(model.summary())

# > Optimization terminated successfully.
#          Current function value: 0.482065
#          Iterations 6
#                            Logit Regression Results
# ==============================================================================
# Dep. Variable:               Survived   No. Observations:                  891
# Model:                          Logit   Df Residuals:                      886
# Method:                           MLE   Df Model:                            4
# Date:                Thu, 20 Jun 2024   Pseudo R-squ.:                  0.2761
# Time:                        14:48:59   Log-Likelihood:                -429.52
# converged:                       True   LL-Null:                       -593.33
# Covariance Type:            nonrobust   LLR p-value:                 1.192e-69
# =====================================================================================
#                         coef    std err          z      P>|z|      [0.025      0.975]
# -------------------------------------------------------------------------------------
# Intercept             0.9466      0.169      5.590      0.000       0.615       1.279
# C(Gender)[T.male]    -2.6422      0.186    -14.197      0.000      -3.007      -2.277
# SibSp                -0.3539      0.098     -3.604      0.000      -0.546      -0.161
# Parch                -0.2007      0.112     -1.792      0.073      -0.420       0.019
# Fare                  0.0147      0.003      5.553      0.000       0.010       0.020
# =====================================================================================

# Parch의 계수: -0.2007
# print(round(-0.2007, 3))
# 2번 답: -0.201

# 3번 로지스틱 회귀모형 SibSp 변수가 한 단위 증가할 때 생존할 오즈비(Odds ratio)
import numpy as np
# print(model.params['SibSp'])

print(np.exp(model.params['SibSp']))
print(round(np.exp(-0.35389193636174865), 3))
print(round(np.exp(-0.3539), 3))

# 3번 답: 0.702


In [ ]:
# 1
df['diff'] = df['bp_after'] - df['bp_before']
result1 = df['diff'].mean()
print(round(result1, 2))

# 2 : 대응표본. 쌍체표본 ttest_rel, alternative 설정
from scipy.stats import ttest_rel

# 대립가설 기준으로 alternative 결정
result = ttest_rel(df['bp_after'], df['bp_before'], alternative='less')

print(result.statistic) # 검정통계량
print(result.pvalue) # pvalue